conda environment for this program:<br>
<code>conda activate stylegan3</code><br>
make sure to include <br>
<code>sys.path.insert(0, "../../../../local_data/packages/stylegan3")</code><br>
to find dnnlib and legacy!

In [1]:
import sys
sys.path.insert(0, "../../../../local_data/packages/stylegan3")
import torch
import dnnlib
import legacy
import PIL.Image
import numpy as np
import imageio
from tqdm.notebook import tqdm

BASE_PATH = "../../../../local_data/jheaton/"
DATA_PATH = "../../../../local_data/tfds/"
PACKAGES_PATH="../../../../local_data/packages/"
SHAPE_PRED_PATH="../../../../local_data/packages/shape_predictor/"
OUTPUT_PATH = BASE_PATH+"class_09_4_facial_points/"
OUT_SOURCE_PATH=OUTPUT_PATH+"out_source/"
OUT_TARGET_PATH=OUTPUT_PATH+"out_target/"

NETWORK = "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl"
STEPS = 150
FPS = 30
FREEZE_STEPS = 30

4.7 Build the Video

In [2]:
lvec1 = np.load(OUT_SOURCE_PATH+'projected_w.npz')['w']
lvec2 = np.load(OUT_TARGET_PATH+'projected_w.npz')['w']

network_pkl = "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl"
device = torch.device('cuda')
with dnnlib.util.open_url(network_pkl) as fp:
    G = legacy.load_network_pkl(fp)['G_ema'].requires_grad_(False).to(device)

diff = lvec2 - lvec1
step = diff / STEPS
current = lvec1.copy()
target_uint8 = np.array([1024,1024,3], dtype=np.uint8)

video = imageio.get_writer(OUTPUT_PATH+'movie.mp4', mode='I', fps=FPS,codec='libx264', bitrate='16M')

for j in tqdm(range(STEPS)):
    z = torch.from_numpy(current).to(device)
    synth_image = G.synthesis(z, noise_mode='const')
    synth_image = (synth_image + 1) * (255/2)
    synth_image = synth_image.permute(0, 2, 3, 1).clamp(0, 255)\
    .to(torch.uint8)[0].cpu().numpy()
    repeat = FREEZE_STEPS if j==0 or j==(STEPS-1) else 1

    for i in range(repeat):
        video.append_data(synth_image)
    current = current + step

video.close()

/tmp/ipykernel_776824/134272979.py:12: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 1024 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)`
will give the desired result (the cast overflows).
  target_uint8 = np.array([1024,1024,3], dtype=np.uint8)


  0%|          | 0/150 [00:00<?, ?it/s]

Setting up PyTorch plugin "bias_act_plugin"... 

/media/data/miniconda3/envs/stylegan3/lib/python3.9/site-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... 

/media/data/miniconda3/envs/stylegan3/lib/python3.9/site-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


Done.
